<a href="https://colab.research.google.com/github/aheiX/Teaching/blob/main/Job%20Shop%20Scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Job Shop Scheduling Problem

Matematical model from: Bülbül, K., & Kaminsky, P. (2013). A linear programming-based method for job shop scheduling. *Journal of Scheduling*, 16, 161-183.

## Notation

\begin{array}{ll}
m & \text{Number of machines}\\
n & \text{Number of jobs}\\
M_j  & \text{Ordered sequence of machines for job } j\\
J_i  & \text{Set of operations to be processed on machine } i\\
p_{ij} & \text{Processing time for job $i$ on machine $i$} \\
r_{j} & \text{Ready time for job } j\\
h_{ij} & \text{Holding cost per unit time for job $j$ while it is waiting in the queue before machine } i\\
w_{ij} & \text{Decision variable (integer) denoting the time job $j$ spends in the queue before machine $i$}\\
C_{ij} & \text{Decision variable (integer) denoting the time at which job $j$ finishes processing on machine $i$}\\
\end{array}
<br>

## Mathematical Model

$
\begin{align}
  \begin{array}{llll}
    &\textbf{Objective} & \\
    & \min \sum\limits_{j=1}^{n} \sum\limits_{i=1}^{m_j} h_{ij} \cdot w_{ij} & &~~~  (1) \\
    &&\\
    &\textbf{Constraints} & \\
    & C_{1j} - w_{1j} = r_j + p_{1j}&,~ \forall~ j=1,...,n  &~~~ (2) \\
    & C_{i-1,j} - C_{ij} + w_{ij} = -p_{ij}&,~ \forall~ j=1,...,n, i=2,...,m_j &~~~ (3) \\
    & C_{ik} - C_{ij} \ge p_{ik}&,~ \forall~ i=1,...,n, i,k \in J_i &~~~ (4a) \\
    & C_{ij} - C_{ik} \ge p_{ij}&,~ \forall~ i=1,...,n, i,k \in J_i &~~~ (4b) \\
    & C_{ij}, w_{ij} \ge 0 &,~ \forall~ j=1,...,n, i=2,...,m_j &~~~ (5) \\
  \end{array}
\end{align}
$
<br><br>
**Important:** Either (4a) OR (4b) not both!

## Python implementation

### Exemplary data

In [24]:
# number of machines
m = [1, 2, 3, 4]

# number of jobs
n = [1, 2]

# [machine] Ordered sequence of machines for a job
M = {1: [1, 3, 4],
     2: [1, 4, 2]}

# [machine][job] Set of jobs on a machine
J = {1: [1, 2],
     2: [2],
     3: [1],
     4: [1, 2]}

# [machine][job] Processing times
p = {1: {1: 6, 2: 2},
     2: {1: 0, 2: 3},
     3: {1: 4, 2: 0},
     4: {1: 1, 2: 9}}


# [job] ready times equals zero for jobs
r = {j: 0 for j in n}

# [machine][job] waiting costs equal one for all machines
h = {i: {j: 1 for j in n} for i in m}

### Packages

In [4]:
!pip install pulp
import pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 69.7 MB/s eta 0:00:00


### Model

In [26]:
# Model
model = pulp.LpProblem(name='Job_Shop_Scheduling', sense=pulp.constants.LpMaximize)

# Decision variables
w = pulp.LpVariable.dicts(name='w', indices=(m, n), lowBound=0, cat='Integer')
C = pulp.LpVariable.dicts(name='C', indices=(m, n), lowBound=0, cat='Integer')

# (1) Objective
model += pulp.lpSum(h[i][j]*w[i][j] for j in n for i in M[j]), '(1)'

# (2)
for j in n:
   model += C[1][j] - w[1][j] == r[j] + p[1][j], '(2)_' + str(j)

# (3)
for j in n:
  for i in M[j]:
    if i != 1:
      model += C[i-1][j] - C[i][j] + w[i][j] == - p[i][j], '(3)_' + str(j) + str(i)

# (4)
for i in m:
  for j in J[i]:
    for k in J[i]:
      # (4a)
      model += C[i][k] - C[i][j] >= p[i][k], '(4a)_' + str(i) + str(j) + str(k)
      # (4b)
      # model += C[i][j] - C[i][k] >= p[i][j], '(4b)_' + str(i) + str(j) + str(k)



### Solution

In [ ]:
# solve problem
model.solve()

# get status
print("Status:", pulp.LpStatus[model.status])

# get objective value
print('Objective value:', round(pulp.value(model.objective), 2))

# get value of decision variable u (position in tour of the nodes that are part of the tour)
for j in n:
  print('Job ' + str(j))
  for i in m:
    if C[i][j].varValue > 0:
      print(' finished at machine ' + str(i) + ' at ' + str(C[i][j]))

Status: Optimal
Objective value: 96.0
Tour: 
 x_node_0_node_2: 1.0, t_ij=17, S_j=73
 x_node_2_node_1: 1.0, t_ij=29, S_j=13
 x_node_1_node_3: 1.0, t_ij=21, S_j=10
 x_node_3_node_5: 1.0, t_ij=14, S_j=6
 total time: 81

u-variable: 
 u_node_1: 3.0
 u_node_2: 2.0
 u_node_3: 4.0
 u_node_4: 2.0
 u_node_5: 6.0
